Импорт и класс HammingCode

In [1]:
!pip install galois

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 36.9 MB/s eta 0:00:00


**Импорт и класс HammingCode**

In [2]:
import numpy as np
import galois
from itertools import product


class HammingCode:
    def __init__(self, q, m):
        """
        q  поле GF(q)
        m  число проверочных символов

        n = (q**m - 1) // (q - 1)
        k = n - m
        """
        self.q = q
        self.m = m
        self.GF = galois.GF(q)

        self.n = (q**m - 1) // (q - 1)
        self.k = self.n - m

        self.H = self._build_H()
        self.G = self._build_G_from_kernel(self.H)
        self._build_syndrome_table()

    # ------------------------------------------------------------
    # Построение проверочной матрицы H
    # ------------------------------------------------------------
    def _build_H(self):
        GF = self.GF
        m = self.m

        reps = []
        seen = set()

        for vec in product(range(self.q), repeat=m):
            if all(v == 0 for v in vec):
                continue

            v = GF(list(vec))

            # нормировка: первый ненулевой приводим к 1
            for i, a in enumerate(v):
                if a != 0:
                    v_norm = v * (GF(1) / a)
                    key = tuple(int(x) for x in v_norm)
                    break

            if key not in seen:
                seen.add(key)
                reps.append(key)

        reps.sort()
        H = GF(np.array(reps).T)
        return H

    # ------------------------------------------------------------
    # Построение G как базиса ядра H
    # ------------------------------------------------------------
    def _build_G_from_kernel(self, H):
        GF = self.GF
        H = GF(H.copy())

        m, n = H.shape
        A = H.copy()

        pivot_rows = []
        pivot_cols = []

        row = 0
        for col in range(n):
            pivot = None
            for r in range(row, m):
                if A[r, col] != 0:
                    pivot = r
                    break
            if pivot is None:
                continue

            if pivot != row:
                A[[row, pivot]] = A[[pivot, row]]

            inv = GF(1) / A[row, col]
            A[row] *= inv

            for r in range(m):
                if r != row and A[r, col] != 0:
                    A[r] -= A[r, col] * A[row]

            pivot_rows.append(row)
            pivot_cols.append(col)
            row += 1
            if row == m:
                break

        free_cols = [j for j in range(n) if j not in pivot_cols]

        basis = []
        for free in free_cols:
            v = GF.Zeros(n)
            v[free] = GF(1)
            for pr, pc in zip(pivot_rows, pivot_cols):
                v[pc] = GF(0) - A[pr, free]
            basis.append(v)

        G = GF(np.vstack(basis))
        return G

    # ------------------------------------------------------------
    # Синдромная таблица
    # ------------------------------------------------------------
    def _build_syndrome_table(self):
        GF = self.GF
        m, n = self.H.shape

        self.syndrome_table = {}

        zero = tuple(int(x) for x in GF.Zeros(m))
        self.syndrome_table[zero] = GF.Zeros(n)

        for pos in range(n):
            for a in range(1, self.q):
                e = GF.Zeros(n)
                e[pos] = GF(a)
                S = (self.H @ e.reshape(-1, 1)).reshape(-1)
                key = tuple(int(x) for x in S)
                if key not in self.syndrome_table:
                    self.syndrome_table[key] = e

    # ------------------------------------------------------------
    # Кодирование
    # ------------------------------------------------------------
    def encode(self, msg):
        GF = self.GF
        msg = GF(msg).reshape(-1)
        if msg.size != self.k:
            raise ValueError(f"Неверная длина сообщения, ожидается {self.k}")
        return msg @ self.G

    # ------------------------------------------------------------
    # Декодирование по синдрому
    # ------------------------------------------------------------
    def decode(self, received):
        GF = self.GF
        r = GF(received).reshape(-1)
        S = (self.H @ r.reshape(-1, 1)).reshape(-1)
        key = tuple(int(x) for x in S)

        e = self.syndrome_table.get(key)
        if e is None:
            return r
        return r - e

    # ------------------------------------------------------------
    # Восстановление сообщения: решаем G^T * m^T = c^T
    # ------------------------------------------------------------
    def decode_to_message(self, received):
        GF = self.GF
        c = self.decode(received).reshape(-1)

        G = self.G
        k, n = G.shape

        A = GF(G.T.copy())           # n x k
        b = GF(c).reshape(-1, 1)     # n x 1

        Ab = GF(np.hstack([A, b]))   # n x (k + 1)

        rows, cols = Ab.shape
        num_vars = k

        r = 0
        pivot_cols = []

        for col in range(num_vars):
            pivot = None
            for i in range(r, rows):
                if Ab[i, col] != 0:
                    pivot = i
                    break
            if pivot is None:
                continue

            if pivot != r:
                Ab[[r, pivot]] = Ab[[pivot, r]]

            inv = GF(1) / Ab[r, col]
            Ab[r] *= inv

            for i in range(rows):
                if i != r and Ab[i, col] != 0:
                    Ab[i] -= Ab[i, col] * Ab[r]

            pivot_cols.append(col)
            r += 1
            if r == rows:
                break

        x = GF.Zeros(num_vars)
        for row_idx, col_idx in enumerate(pivot_cols):
            x[col_idx] = Ab[row_idx, -1]

        return x


# Примеры


In [3]:
print("=== Примеры ===\n")

# ---------------------- GF(2), m=3 ----------------------
hc2 = HammingCode(2, 3)
msg2 = [1, 0, 1, 1]
code2 = hc2.encode(msg2)
r2 = code2.copy()
r2[3] += hc2.GF(1)

print("GF(2), m=3")
print("Сообщение:              ", msg2)
print("Кодовое слово:          ", np.array(code2, dtype=int))
print("Принято с ошибкой:      ", np.array(r2, dtype=int))
print("Исправленное слово:     ", np.array(hc2.decode(r2), dtype=int))
print("Восстановленное сообщение:", np.array(hc2.decode_to_message(r2), dtype=int))
print("\n")


# ---------------------- GF(3), m=2 ----------------------
hc3 = HammingCode(3, 2)
msg3 = [1, 2]
code3 = hc3.encode(msg3)
r3 = code3.copy()
r3[1] += hc3.GF(2)

print("GF(3), m=2")
print("Сообщение:              ", msg3)
print("Кодовое слово:          ", np.array(code3, dtype=int))
print("Принято с ошибкой:      ", np.array(r3, dtype=int))
print("Исправленное слово:     ", np.array(hc3.decode(r3), dtype=int))
print("Восстановленное сообщение:", np.array(hc3.decode_to_message(r3), dtype=int))
print("\n")


# ---------------------- GF(5), m=2 ----------------------
hc5 = HammingCode(5, 2)
msg5 = [3, 4, 1, 0]
code5 = hc5.encode(msg5)
r5 = code5.copy()
r5[2] += hc5.GF(3)

print("GF(5), m=2")
print("Сообщение:              ", msg5)
print("Кодовое слово:          ", np.array(code5, dtype=int))
print("Принято с ошибкой:      ", np.array(r5, dtype=int))
print("Исправленное слово:     ", np.array(hc5.decode(r5), dtype=int))
print("Восстановленное сообщение:", np.array(hc5.decode_to_message(r5), dtype=int))


=== Примеры ===

GF(2), m=3
Сообщение:               [1, 0, 1, 1]
Кодовое слово:           [0 1 1 0 0 1 1]
Принято с ошибкой:       [0 1 1 1 0 1 1]
Исправленное слово:      [0 1 1 0 0 1 1]
Восстановленное сообщение: [1 0 1 1]


GF(3), m=2
Сообщение:               [1, 2]
Кодовое слово:           [1 0 1 2]
Принято с ошибкой:       [1 2 1 2]
Исправленное слово:      [1 0 1 2]
Восстановленное сообщение: [1 2]


GF(5), m=2
Сообщение:               [3, 4, 1, 0]
Кодовое слово:           [1 2 3 4 1 0]
Принято с ошибкой:       [1 2 1 4 1 0]
Исправленное слово:      [1 2 3 4 1 0]
Восстановленное сообщение: [3 4 1 0]
